# FAST-UAV - Uncertainty analysis

The purpose of this notebook is to run uncertainty analyses in order to evaluate the impacts of models uncertainty and input parameters uncertainty.

## 1. Setting up the problem

In a first place, we define in the configuration file the UAV models that will be used to run the analyses. Here, no optimization of the design is carried out, i.e. we start from an existing design and analyse its performance on a given mission.

In [1]:
import os.path as pth
import openmdao.api as om
import logging
import shutil
import fastoad.api as oad
from time import time
from openmdao_drivers.cmaes_driver import CMAESDriver
from nrel_openmdao_extensions.nlopt_driver import NLoptDriver
import cma
import matplotlib.pyplot as plt
from utils.postprocessing.sensitivity_analysis.sensitivity_analysis import (
    morris_analysis,
    sobol_analysis,
)

plt.rcParams["figure.figsize"] = 16, 8
plt.rcParams.update({"font.size": 13})

DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"

# For having log messages display on screen
# logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

D:\Softwares\Anaconda3\envs\OAD\lib\site-packages\pyoptsparse\pyOpt_MPI.py:68: UserWarning: mpi4py could not be imported. mpi4py is required to use the parallel gradient analysis and parallel objective analysis for non-gradient based optimizers. Continuing using a dummy MPI module from pyOptSparse.
  warnings.warn(warn)


Let's start by defining two problems that will be useful for the uncertainty analysis:

- The first problem consists of the UAV model alone. In this problem, no optimization of any kind is carried out and the model is run once.
- The second problem is made of the UAV model on which a consistency optimization in run. That is, the system consistency is guaranteed even if the inputs parameters of the problem are modified. For more information about the consistency constraints please refer to Delbecq et al. article available [here](https://oatao.univ-toulouse.fr/26691/).

These problems are deterministic, i.e. they take as inputs fixed values and return deterministic outputs.

In [2]:
CONF_FILE = pth.join(DATA_FOLDER_PATH, "multirotor_model.yaml")  # Problem without consistency constraint solving
CONF_FILE_CONSISTENT = pth.join(DATA_FOLDER_PATH, "multirotor_model_consistency.yaml")  # Problem with consistency constraints solving

In [3]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONF_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width="100%", height="500px")

We now load an existing UAV design on which the uncertainty analysis will be run.

In [4]:
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "problem_outputs_DJI_M600_mdo.xml")
oad.variable_viewer(SOURCE_FILE)

To make sure that the source file is consistent with the problem definition and the models, a simple run is carried out. No difference between the values of the source file and the output values of the problem should be observed.<br>
We will continue this notebook with the output values of the deterministic problem. These deterministic values are stored in a separate XML file.

In [5]:
INPUT_FILE = oad.generate_inputs(CONF_FILE, SOURCE_FILE, overwrite=True)
# problem = oad.optimize_problem(CONF_FILE, overwrite=True)  # Run model once (no optimization)
problem = oad.optimize_problem(CONF_FILE_CONSISTENT, overwrite=True)  # Run model with consistency constraints
DETERMINISTIC_FILE = pth.join(WORK_FOLDER_PATH, "model_outputs.xml")
oad.variable_viewer(DETERMINISTIC_FILE)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5328299458777948
            Iterations: 1
            Function evaluations: 1
            Gradient evaluations: 1
Optimization Complete
-----------------------------------


## 2. Design of Experiments

In contrast with a single problem evaluation, the uncertainty analysis relies on multiple evaluations of the problem to analyse the effects of varying one or several parameters on the output variables of interest.

The [OpenTurns](https://openturns.github.io/openturns/latest/index.html) and [SALib](https://salib.readthedocs.io/en/latest/index.html) libraries are used to set up the DoEs and run the sensitivity analyses. 
The drivers to connect these librairies to FAST-UAV are provided by Onera's [OpenMDAO Extensions](https://github.com/OneraHub/openmdao_extensions).

Keep in mind that two distinct approaches for the uncertainty analysis are possible:
- in the first approach (*'CONF_FILE'*), a single straightforward calculation of the models is carried out;
- in the approach (*'CONF_FILE_CONSISTENT'*), the driver will iterate over the consistency variables to ensure system consistency.

## 2.1 Screening procedure - Morris method

The Morris method allows to get a measure of importance and interaction of input factors. It is used as a screening method to reduce the number of parameters prior to a more detailed analysis.

The `morris_analysis` function takes the following inputs:
- The configuration file of the problem to be evaluated;
- The source file containing the deterministic design, to help the user to set up the analysis (uncertain parameters selection, output variable of interest, ...). 

The number of simulation runs required are $r(k+1)$ where $r$ is the number of trajectories (typically between 4 to 10) and $k$ the number of parameters.

**Bibliography**

> Ruano, M.V., Ribes, J., Seco, A., Ferrer, J., 2012. An improved sampling strategy based on trajectory design for application of the Morris method to systems with many input factors. Environmental Modelling & Software 37, 103–109. https://doi.org/10.1016/j.envsoft.2012.03.008

> Morris, M.D., 1991. Factorial Sampling Plans for Preliminary Computational Experiments. Technometrics 33, 161–174. https://doi.org/10.1080/00401706.1991.10484804

> Campolongo, F., Cariboni, J., Saltelli, A., 2007. An effective screening design for sensitivity analysis of large models. Environmental Modelling & Software, Modelling, computer-assisted simulations, and mapping of dangerous phenomena for hazard assessment 22, 1509–1518. https://doi.org/10.1016/j.envsoft.2006.10.004


In [6]:
# morris_analysis(CONF_FILE, DETERMINISTIC_FILE)
morris_analysis(CONF_FILE_CONSISTENT, DETERMINISTIC_FILE)

    'data': [{'error_y': {'array': [], 'type': 'data'},
         …

## 2.2 Global sensitivity analysis - Sobol' indices using Saltelli's sampling scheme

The Sobol' method allows to get a representation of the contribution of the inputs to the overall uncertainty in the model output. The space of the uncertain inputs is explored with a Monte Carlo sampling (here Saltelli's scheme, which extends the Sobol' sequence in a way to reduce the error rates in the resulting sensitivity index calculations).

The number of simulation runs required are $m(k+2)$ where $m$ is the number of samples (typically between 100s to 1000s) and $k$ the number of parameters.

**Bibliography**

> Campolongo, F., Saltelli, A., Cariboni, J., 2011. From screening to quantitative sensitivity analysis. A unified approach. Computer Physics Communications 182, 978–988. https://doi.org/10.1016/j.cpc.2010.12.039

> Saltelli, A., 2002. Making best use of model evaluations to compute sensitivity indices. Computer Physics Communications 145, 280–297. https://doi.org/10.1016/S0010-4655(02)00280-1

> Sobol’, I.M., 2001. Global sensitivity indices for nonlinear mathematical models and their Monte Carlo estimates. Mathematics and Computers in Simulation, The Second IMACS Seminar on Monte Carlo Methods 55, 271–280. https://doi.org/10.1016/S0378-4754(00)00270-6


In [7]:
# sobol_analysis(CONF_FILE, DETERMINISTIC_FILE)
sobol_analysis(CONF_FILE_CONSISTENT, DETERMINISTIC_FILE)

    'data': [{'error_y': {'array': [], 'type': 'data'},
         …

## 2.3 -  Expert mode
You may reuse data from the DoEs ran with `morris_analysis` and `sobol_analysis` methods to display custom plots or run new analyses.

In [ ]:
import pandas as pd
import seaborn as sns
from utils.postprocessing.sensitivity_analysis.sobol_plot import sobol_plot
from utils.postprocessing.sensitivity_analysis.morris_plot import morris_plot
from SALib.analyze import sobol, morris

plt.rcParams.update({"font.size": 20, "lines.markersize": 10})
SA_PATH = "./workdir/sensitivity_analysis"

In [ ]:
# MORRIS METHOD

# Load DoE csv file
df_morris = pd.read_csv(SA_PATH + "/doe_Morris.csv", sep=",")

# Load problem and output definitions
f_problem = open(SA_PATH + "/problem_morris.txt", "r")
f_inputs = open(SA_PATH + "/x_morris.txt", "r")
f_output = open(SA_PATH + "/y_morris.txt", "r")
# print(f_problem.read())
problem = eval(f_problem.read())
x = eval(f_inputs.read())
y = f_output.read()
f_problem.close()
f_inputs.close()
f_output.close()

# Read data
X = df_morris[x].to_numpy()
Y = df_morris[y].to_numpy()

# Run Morris analysis with SALib
Si = morris.analyze(
    problem, X, Y, print_to_console=False, conf_level=0.95, num_resamples=100
)

# Plot results
fig = morris_plot(Si, unit="(min)")
plt.savefig(SA_PATH + "/figures/morris_plot.pdf", bbox_inches="tight")

In [ ]:
# SOBOL' METHOD

# Load DoE csv file
df_sobol = pd.read_csv(SA_PATH + "/doe_Sobol.csv", sep=",")

# Load problem and output definitions
f_problem = open(SA_PATH + "/problem_sobol.txt", "r")
f_output = open(SA_PATH + "/y_sobol.txt", "r")
problem = eval(f_problem.read())
y = f_output.read()
f_problem.close()
f_output.close()

# Read data
Y = df_sobol[y].to_numpy()

# Run Sobol analysis with SALib
Si = sobol.analyze(problem, Y, calc_second_order=True)

# Plot results
fig = sobol_plot(Si)
plt.savefig(SA_PATH + "/figures/sobol_plot.pdf", bbox_inches="tight")

In [ ]:
# OUTPUT DISTRIBUTION
from scipy.stats import norm
import numpy as np

df_output = pd.read_csv(SA_PATH + "/doe_Sobol.csv", sep=",")
mu, std = norm.fit(df_output[y])
print("mu = ", mu)
print("std = ", std)

# Plot the histogram.
q25, q75 = np.percentile(df_output[y], [25, 75])
bin_width = (
    2 * (q75 - q25) * len(df_output[y]) ** (-1 / 3)
)  # Freedman–Diaconis number of bins
bins = round((df_output[y].max() - df_output[y].min()) / bin_width)
plt.hist(
    df_output[y],
    bins=bins,
    density=True,
    alpha=1.0,
    rwidth=0.85,
    label="output distribution",
)

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, "r", linewidth=5, label="normal distribution")

plt.xlabel("Hover Autonomy (min)")
plt.ylabel("Probability")
plt.legend()

# fig = sns.displot(df_sobol, x=y, kde=True, stat="probability", binwidth=0.5, aspect=1.5)
# fig.set_axis_labels("Hover autonomy (min)", "Probability")

plt.savefig(SA_PATH + "/figures/output_dist.pdf", bbox_inches="tight")